# Model Deployment with CivisML

If you are familiar with scikit-learn and/or CivisML, deploying machine learning models in Python can now be done in Civis Platform with just a couple of API calls. This notebook will walk you through the following steps:

1. Training a model on some toy data in CivisML,
2. Deploying that model as a web service on Civis Platform,
3. Getting the URL and an access token for your deployed model, and
4. Making requests of your now-deployed model. 

By executing the code in the cells below, you will be able to get predictions from a deployed model in about 5 minutes. 

In [ ]:
# NOTE: you should be using version 1.8.X or 1.9.X of the Civis API client
import civis

### First, make some toy data

Let's suppose we want to make a model to determine whether an individual will likely be persuaded a particluar message. For each person in our (artificial) training data set, we have `age`, `gender`, and political `party` as features. We also have binary labels `msg` for whether or not the individual said they liked a message we showed them.

In [ ]:
from io import StringIO
import pandas as pd
data = 'age,gender,party,msg\n20,M,other,1\n25,M,D,0\n30,F,D,0\n40,M,R,1\n50,F,D,0\n60,M,other,1\n65,F,R,0\n70,F,D,1\n80,M,R,1\n'

df = pd.read_csv(StringIO(data))

The first five rows in our training data look like:

In [ ]:
df.head()

### Next, let's build a model with CivisML

Using CivisML, we can train a random forest classifier to predict the probability that future individuals will also respond positively to the message.

In [ ]:
mp = civis.ml.ModelPipeline(model='random_forest_classifier', dependent_variable='msg')
_ = mp.train(df)

In [ ]:
mp.train_result_.train_job_id

You can monitor the status of your training job on Platform:

https://platform.civisanalytics.com/#/

Your model must finish training before you can deploy it.  

### We can deploy this model with two API calls

When the model has finished training, we can create a service for deployment by passing the training job ID (`mp.train_result_.train_job_id`) as an environment variable to our CivisML model deployment Docker image. You can optionally pass in the training run ID as well (`mp.train_result_.train_run_id`); if no such run ID is provided, the most recent run is used by default. 

In [ ]:
client = civis.APIClient(resources='all')

# The version of the Civis API client with which we trained our model determines which
# Docker image we should use. 
civ_minor_version = '.'.join(civis.__version__.split('.')[:2])
if civ_minor_version == '1.9':
    image_tag = '1.1'
elif civ_minor_version == '1.8':
    image_tag = '1.0'
else:
    print("WARNING: Your model may not properly deploy. Your version of the Civis "
          "API client is {}, but it should be either v1.8 or 1.9!".format(civis.__version__))
    image_tag = '1.0'

resp = client.services.post(
    name='my_deployed_model',
    docker_image_name="civisanalytics/model-deployment",
    docker_image_tag=image_tag,
    cpu=1000,
    memory=8000, 
    environment_variables={'MODEL_JOB_ID': mp.train_result_.train_job_id, 
                           'DEBUG': 1}   # The optional "DEBUG" environment variable turns on verbose logging
)

The `DEBUG` environment variable ensures that our logs are verbose. You may want this turned off for models in production.

Once we have created our service, we can start the deployment with the following API call:

In [ ]:
_ = client.services.post_deployments(resp['id'])

It may take a minute or two for the deployment to start up, but once it does we can make requests from our model. You can monitor the logs of your deployment on Platform.

In [ ]:
print("Platform page for your deployment: ")
print("https://platform.civisanalytics.com/#/services/{}".format(resp['id']))

You can find the logs under the "Deployment History" link towards the upper right-hand side of the screen. When the logs say "_Application successfully changed to ready state_" you're ready to make calls.

### Let's get some predictions from our model!

First, we'll need to get the model's URL and an access token. Once we have those, we can use python's `requests` library to make some simple RESTful calls. 

In [ ]:
url = client.services.get(resp['id'])['current_url']
print(url)

In [ ]:
token_resp = client.services.post_tokens(resp['id'], 'keiths_token')

We can get predictions from our model by passing covariates as query parameters to the `/predict` endpoint and making a GET call:

In [ ]:
import requests

# Put the token in the header of the HTTP call
headers = {"Authorization": "Bearer {}".format(token_resp['token'])}
# Pass your model covariates as query parameters
pred_url = url + '/predict?age=30&gender=F&party=D'

# Make the GET call
getresp = requests.get(pred_url, headers=headers)
print(getresp.text)

Look: a prediction! Predictions are returned as JSON dictionaries. 

Instead of passing our access token through the HTTP header, we can optionally pass it as a query parameter using the `civis_service_token` keyword:

In [ ]:
pred_url = url + '/predict?age=30&gender=F&party=D&civis_service_token=' + token_resp['token']

# Make the GET call, no headers necessary
getresp = requests.get(pred_url)
print(getresp.text)

### Model GUI

Go to your model's deployment page:

In [ ]:
print("https://platform.civisanalytics.com/#/services/{}".format(resp['id']))

Clicking on the "View Report" button towards the upper right-hand side of the screen will take you to a web form where you can enter model covariates and get predictions interactively. Try it!